In [1]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('../../code/')
import leakage
import utils
import FBA_leak

In [2]:
from dotenv import find_dotenv

In [3]:
model = cobra.io.read_sbml_model('../../models/e_coli/momentiJO1366.xml')
model.solver = 'gurobi'

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [4]:
model.reactions.DHAPT.knock_out()
model.reactions.THRA.knock_out()
model.reactions.THRA2.knock_out()
model.reactions.F6PA_TG_forward.knock_out()

In [5]:
model.reactions.EX_glc__D_e.lower_bound = -10


In [6]:
# Initial conditions is 0.013 gDW/L (in total)
model_name = 'Ecoli'
model_name_dict = {model_name: [model, 0.006]}

glucose_mM = utils.convert_gL_to_mM("C6H12O6", 4)
D = FBA_leak.dFBA(iterations = 3, dt = 0.1, method = "FBA_with_leakage", store_exchanges_flag = False)#(, pfba_fraction = 0.95)
D.add_models(model_name_dict)

Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmp3ay9f5d_.lp
Reading time = 0.01 seconds
: 1806 rows, 6356 columns, 29136 nonzeros


In [7]:
m = model.metabolites.prot_pool

In [9]:
# Set Km and vMax
D.models[model_name].set_km("glc__D_e", 1)
D.models[model_name].set_Vmax("glc__D_e", 10)



D.medium.define_initial_conditions({"glc__D_e": glucose_mM})
# D.medium.set_store_concentrations(["glc__D_e", "nh3_e"])
D.run()

KeyError: 'DM_4crsol_c'